In [370]:
import pandas as pd
import numpy as np

# Input file name
filename = 'priceTest.csv'


In [371]:
# Read the input file into a pandas DataFrame
df = pd.read_csv(filename)


In [372]:
# Generate a sequence of unique numbers using numpy
unique_numbers = np.arange(1, len(df) + 1)

In [373]:

# Assign the sequence of unique numbers to a new column in the DataFrame
df['ID_Number'] = unique_numbers


In [374]:
#Rename IP Expected Reimbursement and OP Expected Reimbursement to just IP and OP. 
# This will be appended into each Plans. This will resulce header length instead of the whole IP Expected Reimbursement or OP Expected Reimbursement
df_Rename = df.rename(columns={'IP Expected Reimbursement': 'IP', 'OP Expected Reimbursement': 'OP'})


In [375]:
# Replace spaces in column names with underscores
df_Rename.columns = [col.replace(' ', '_') for col in df_Rename.columns]


In [376]:
# Select the columns to include in the output
df_Rename = df_Rename[['ID_Number', 'Code_type', 'Code', 'Procedure_Description', 'NDC', 'Rev_Code', 'IP_Price', 'OP_Price', 'Plan(s)', 'IP', 'OP']]
print(df_Rename.shape)
print(df_Rename)


(4, 11)
   ID_Number Code_type       Code Procedure_Description  NDC  \
0          1       EAP  CPT 10080           Cyst Simple  NaN   
1          2       EAP  CPT 10080           Cyst Simple  NaN   
2          3       EAP  CPT 10080           Cyst Simple  NaN   
3          4       EAP  CPT 10080           Cyst Simple  NaN   

                                            Rev_Code  IP_Price  OP_Price  \
0                 0516 - CLINIC - URGENT CARE CLINIC       100       100   
1  0516 - CLINIC - EMERGENCY ROOM URGENT CARE CLINIC       100       100   
2                 0516 - CLINIC - URGENT CARE CLINIC       100       100   
3                 0516 - CLINIC - URGENT CARE CLINIC       100       100   

       Plan(s)   IP   OP  
0     Self Pay  200  200  
1  Payer1Plan1  300  400  
2  Payer1Plan2  250  350  
3  Payer1Plan2  100  350  


In [377]:
output_filename = 'ColumnSelected.csv'

# Write the output to a CSV file
df_Rename.to_csv(output_filename, index=False)

# Use R code to perform aggregation per Plans

In [ ]:
# Place R code here

In [416]:
# Change to Python. Use the aggregated output from R
# Input filename from R
RfileName = 'PT_Grouped.csv'

# Read the input file into a pandas DataFrame
PT = pd.read_csv(RfileName)


In [417]:
# Generate another sequence of unique numbers using numpy
unique_number2 = np.arange(1, len(PT) + 1)

In [418]:
# Assign the sequence of unique numbers to a new column in the DataFrame
PT['ID_No'] = unique_number2

In [419]:
print(PT)

  Code_type       Code Procedure_Description  NDC  \
0       EAP  CPT 10080           Cyst Simple  NaN   
1       EAP  CPT 10080           Cyst Simple  NaN   
2       EAP  CPT 10080           Cyst Simple  NaN   

                                            Rev_Code        Plans  IP_Price  \
0  0516 - CLINIC - EMERGENCY ROOM URGENT CARE CLINIC  Payer1Plan1       100   
1                 0516 - CLINIC - URGENT CARE CLINIC  Payer1Plan2       100   
2                 0516 - CLINIC - URGENT CARE CLINIC     Self Pay       100   

   OP_Price   IP   OP  ID_No  
0       100  300  400      1  
1       100  175  350      2  
2       100  200  200      3  


In [420]:
# Pivot the DataFrame on the 'Plan(s)' column
PT = PT.pivot(index=['Code_type', 'Code', 'Procedure_Description', 'NDC', 'Rev_Code',  'IP_Price', 'OP_Price'], columns='Plans', values=['IP', 'OP'])
print(PT)

                                                                                                                           IP  \
Plans                                                                                                             Payer1Plan1   
Code_type Code      Procedure_Description NDC Rev_Code                                          IP_Price OP_Price               
EAP       CPT 10080 Cyst Simple           NaN 0516 - CLINIC - EMERGENCY ROOM URGENT CARE CLINIC 100      100            300.0   
                                              0516 - CLINIC - URGENT CARE CLINIC                100      100              NaN   

                                                                                                                               \
Plans                                                                                                             Payer1Plan2   
Code_type Code      Procedure_Description NDC Rev_Code                                          

In [421]:
# Flatten the column names in the pivot table
PT.columns = PT.columns.map('_'.join)


In [422]:
PT.columns = PT.columns.map(lambda x: x.split('_')[::-1]).map('_'.join)

In [423]:
# Reset the index to include the original columns in the output
PT = PT.reset_index()

In [424]:
# Get the subset of columns to sort
cols_to_sort = [col for col in PT.columns if '_IP' in col or '_OP' in col]

# Sort the subset of columns alphabetically
PT = PT.reindex(columns=sorted(cols_to_sort) + [col for col in PT.columns if col not in cols_to_sort])


In [425]:
# Get the columns to keep in front
cols_to_keep_in_front = ['Code_type' ,'Code','Procedure_Description', 'NDC', 'Rev_Code', 'IP_Price', 'OP_Price']

In [426]:
# Sort the subset of columns alphabetically
PT = PT[cols_to_keep_in_front + sorted(cols_to_sort)]


In [427]:
# Get the minimum and maximum values for the columns with '_IP'
cols_ip = [col for col in PT.columns if '_IP' in col]
min_ip = PT[cols_ip].min(axis=1)
max_ip = PT[cols_ip].max(axis=1)

# Get the minimum and maximum values for the columns with '_OP'
cols_op = [col for col in PT.columns if '_OP' in col]
min_op = PT[cols_op].min(axis=1)
max_op = PT[cols_op].max(axis=1)

In [428]:
PT.loc[:, 'MIN_IP'] = min_ip
PT.loc[:, 'MAX_IP'] = max_ip
PT.loc[:, 'MIN_OP'] = min_op
PT.loc[:, 'MAX_OP'] = max_op


In [429]:
print(PT)

  Code_type       Code Procedure_Description  NDC  \
0       EAP  CPT 10080           Cyst Simple  NaN   
1       EAP  CPT 10080           Cyst Simple  NaN   

                                            Rev_Code  IP_Price  OP_Price  \
0  0516 - CLINIC - EMERGENCY ROOM URGENT CARE CLINIC       100       100   
1                 0516 - CLINIC - URGENT CARE CLINIC       100       100   

   Payer1Plan1_IP  Payer1Plan1_OP  Payer1Plan2_IP  Payer1Plan2_OP  \
0           300.0           400.0             NaN             NaN   
1             NaN             NaN           175.0           350.0   

   Self Pay_IP  Self Pay_OP  MIN_IP  MAX_IP  MIN_OP  MAX_OP  
0          NaN          NaN   300.0   300.0   400.0   400.0  
1        200.0        200.0   175.0   200.0   200.0   350.0  


In [430]:
# Output file name
output_filename = 'PriceTransparency.csv'

# Write the output to a CSV file
PT.to_csv(output_filename, index=False)